# Feature extraction

# Step1
import dependencies

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

# Step2
load the payloads into memory

In [5]:
payloads = pd.read_csv("data/payloads.csv",index_col='index')
display(payloads.head(30))

,payload,is_malicious,injection_type
index,,,
0,abellot*ada,0.0,LEGAL
1,1280x800,0.0,LEGAL
2,Juergen,0.0,LEGAL
3,ambitious,0.0,LEGAL
4,Shalne,0.0,LEGAL
5,joann,0.0,LEGAL
6,tonnage,0.0,LEGAL
7,Kowtow,0.0,LEGAL
8,3683134199988856,0.0,LEGAL


# Step3
feature engineering

In [11]:
def create_feature_example(payloads):
    '''small description
    
    longer description
    '''
    
    payloads['example'] = [1 for row in payloads['payload']]
    return payloads

payloads = create_feature_example(payloads)
display(payloads.head())



,payload,is_malicious,injection_type,example
index,,,,
0,abellot*ada,0.0,LEGAL,1
1,1280x800,0.0,LEGAL,1
2,Juergen,0.0,LEGAL,1
3,ambitious,0.0,LEGAL,1
4,Shalne,0.0,LEGAL,1
